In [32]:
import cv2
import requests
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import logging
import absl.logging

# Suppress TensorFlow warnings
import warnings
warnings.filterwarnings("ignore")

# Suppress TensorFlow-specific logging messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING messages (0 = all, 1 = info, 2 = warnings, 3 = errors)

# Correct method for TensorFlow 2.x to suppress deprecation warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [33]:
# Suppress absl warnings
logging.getLogger("absl").setLevel(logging.ERROR)

# Load the pre-trained EfficientDet model
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/d2/1")

In [34]:
class_to_ingredient = {
    1: "tomato",
    2: "onion",
    3: "potato",
    4: "garlic",
    5: "ginger",
    6: "green chili",
    7: "coriander leaves (cilantro)",
    8: "mint leaves",
    9: "curry leaves",
    10: "spinach (palak)",
    11: "paneer",
    12: "cumin seeds",
    13: "mustard seeds",
    14: "turmeric powder",
    15: "red chili powder",
    16: "garam masala",
    17: "coriander powder",
    18: "fennel seeds",
    19: "fenugreek seeds",
    20: "cardamom",
    21: "cloves",
    22: "cinnamon",
    23: "bay leaves",
    24: "asafoetida (hing)",
    25: "ghee",
    26: "rice",
    27: "wheat flour",
    28: "biryani masala",
    29: "chicken",
    30: "mutton",
    31: "fish",
    32: "eggs",
    33: "chana dal",
    34: "toor dal",
    35: "moong dal",
    36: "masoor dal",
    37: "urad dal",
    38: "coconut",
    39: "coconut milk",
    40: "jaggery",
    41: "tamarind",
    42: "lemon",
    43: "yogurt (curd)",
    44: "buttermilk",
    45: "milk",
    46: "cream",
    47: "cashews",
    48: "almonds",
    49: "raisins",
    50: "bitter gourd (karela)",
    51: "brinjal (eggplant)",
    52: "cauliflower",
    53: "capsicum (bell pepper)",
    54: "ladyfinger (okra)",
    55: "cabbage",
    56: "carrot",
    57: "green peas",
    58: "methi leaves (fenugreek)",
    59: "mooli (radish)",
    60: "gourd (lauki)",
    61: "pumpkin",
    62: "kala chana (black chickpeas)",
    63: "rajma (kidney beans)",
    64: "black gram (urad dal whole)",
    65: "poha (flattened rice)",
    66: "sooji (semolina)",
    67: "besan (gram flour)",
    68: "sabudana (tapioca pearls)",
    69: "mustard oil",
    70: "vegetable oil",
    71: "saffron",
    72: "poppy seeds",
    73: "whole wheat",
    74: "puffed rice",
    75: "pickle",
    76: "papad",
    77: "jowar",
    78: "bajra",
    79: "amchur (dried mango powder)",
    80: "khoya",
    81: "makhana (fox nuts)",
    82: "tandoori masala",
    83: "chaat masala",
    84: "pav bhaji masala",
    85: "sambar powder",
    86: "rasam powder",
    87: "idli rice",
    88: "black salt",
    89: "kasuri methi (dried fenugreek leaves)",
    90: "imli (tamarind)",
    91: "soya chunks",
    92: "gond (edible gum)",
    93: "badam (almond)",
    94: "kaju (cashew)",
    95: "dahi (curd)"
}


In [35]:
def capture_image():
    cap = cv2.VideoCapture(0)  # 0 is usually the primary webcam

    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imshow('Capture Image', frame)  # Display the frame

            # Press 's' to save the captured image
            if cv2.waitKey(1) & 0xFF == ord('s'):
                img_name = "captured_image.jpg"
                cv2.imwrite(img_name, frame)
                print(f"Image saved as {img_name}")
                break

    cap.release()
    cv2.destroyAllWindows()

    return img_name

In [36]:
def detect_ingredients(image_path):
    # Load and preprocess image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = tf.image.resize(image_rgb, (512, 512))
    image_resized = np.expand_dims(image_resized, axis=0)
    
    # Detect objects
    detections = detector(image_resized)

    # Extract detection classes and confidence scores
    class_ids = detections['detection_classes'][0].numpy().astype(int)
    scores = detections['detection_scores'][0].numpy()

    print("Class IDs:", class_ids)
    print("Scores:", scores)

    return class_ids, scores

In [40]:
def get_detected_ingredients(class_ids, scores, class_to_ingredient):
    threshold = 0.3  # Set the threshold for detection
    detected_ingredients = []

    # Iterate through the detections and map to ingredients
    for c, s in zip(class_ids, scores):
        if s > threshold:  # Filter by confidence threshold
            ingredient = class_to_ingredient.get(c, None)
            if ingredient:  # Only add valid ingredients
                detected_ingredients.append(ingredient)
    
    return detected_ingredients

In [41]:
def get_recipes(ingredients):
    api_key = '7e5a62c0114144bebabeefcdaacc68ac'  
    url = f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={','.join(ingredients)}&number=5&apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        recipes = [recipe['title'] for recipe in data]
        return recipes
    else:
        print("Error fetching recipes from API:", response.status_code)
        return []

In [42]:
# Main sequence
image_path = capture_image()  # Capture image from webcam

# Detect ingredients from the captured image
class_ids, scores = detect_ingredients(image_path)

# Get the detected ingredients
detected_ingredients = get_detected_ingredients(class_ids, scores, class_to_ingredient)

# If ingredients are detected, fetch recipes based on them
if detected_ingredients:
    print("Detected Ingredients:", detected_ingredients)
    
    # Fetch recipes based on detected ingredients
    recipes = get_recipes(detected_ingredients)
    
    if recipes:
        print("\nHere are some recipe suggestions based on your ingredients:")
        for i, recipe in enumerate(recipes, 1):
            print(f"{i}. {recipe}")
    else:
        print("No recipes found.")
else:
    print("No ingredients detected in the image.")

Image saved as captured_image.jpg
Class IDs: [53 53 53 51 55 78 72 73 53 53 47  1 64 55 67 77 60 85 44 79 82  1 70 53
 84 55 53 55  1 86 80 88 61 37 53 16 53 17 44 44 62 86 74 16 67 60 53 18
 28 51 51 67 60 52 53  3 86 33 53 53 84 53 10 53 61 55 31 18 44 51 81 60
 52 64  1 57 11 59 67 53 37 57 17 85 59 13 57 16 53 47 44 64 86 55 53 75
 22 85 76 53]
Scores: [0.37221915 0.34450647 0.29067925 0.19707213 0.1609007  0.1502255
 0.13888572 0.1237061  0.10748978 0.10735993 0.10698913 0.09092896
 0.08981152 0.08821853 0.07563001 0.07268135 0.07263073 0.07063788
 0.07017178 0.06882948 0.06817944 0.06550728 0.06330455 0.06325056
 0.06220949 0.06127078 0.06007133 0.05775639 0.05621123 0.05363125
 0.05353758 0.0520744  0.04959127 0.04809403 0.04796681 0.04757167
 0.04744935 0.04739687 0.04710542 0.04687952 0.04557781 0.04550155
 0.04513703 0.04483922 0.04439805 0.04396169 0.04284529 0.04237014
 0.04102265 0.04043442 0.04036237 0.04027218 0.03941602 0.03896592
 0.03894014 0.03881106 0.03848684 0.037